# Regional mean differences and GWLs

This notebook calculates regional mean differences of SST and STAS with respect to global mean temperature, and GWL for each of the 26 global climate model

Loading libraires:

In [1]:
import netCDF4 as nc
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
from scipy import signal
import warnings
import os
import sys

Ignore warnings:

In [2]:
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.simplefilter("ignore", category=RuntimeWarning)

Define enviromental variables and paths:

In [3]:
season = 'SON' 
resolution = 'low_res'
mask_file    = 'ipcc_regions_05.nc' 
member_names = f'models_{resolution}.txt' 
varname      = 'tos'
ssps         = ['ssp126', 'ssp245', 'ssp370', 'ssp585']
avg_period   = 10 # years
ref_ntimes   = 51 # not including years from 1900-1950
ystart_to_plot  = 2010

root = '..'
path2info  = f'{root}/data/data_info/'
path2data  = f'{root}/data/data_raw/{resolution}/{varname}/'  
path2mask  = f'{root}/data/masks/'

Define functions

In [4]:
def read_txt(fname, ncolumns):
    if ncolumns > 1:
        fdata   = open(fname)
        flines  = np.array(fdata.readlines())
        data_in = np.empty([len(flines), ncolumns])
        for i in np.arange(len(flines)):
            data_in[i,:]=flines[i].split()
    else:
        with open(fname) as flist:
            data_in = np.array(flist.readlines())        
    return(data_in)
    fdata.close()

def read_nc(fname, varname, ifmerged):
    if ifmerged:
        f  = nc.MFDataset(fname)
    else:
        f  = nc.Dataset(fname)
    var    = f.variables[varname]
    lon    = f.variables['lon']
    lat    = f.variables['lat']
    time   = f.variables['time']
    dates  = nc.num2date(time[:], time.units, time.calendar)
    return var, lon, lat, time, dates

def read_mask(fname, varname):
    f = nc.Dataset(fname)
    var  = f.variables[varname][:]
    return var

def robust(mean_hist, mean_future, avg_period):
    mean_hist_detrend = signal.detrend(mean_hist)
    treshold = np.sqrt(2)*1.645*mean_hist_detrend.std()/np.sqrt(avg_period)
    if abs(mean_future) > treshold:
        data_robust = 1
    else:
        data_robust = 0
    return data_robust

In [5]:
member_list = f'{path2info}/{member_names}'
members  = read_txt(member_list, 1)
nmembers = len(members)

Define the mask variable for corresponding masks, and load the mask:

In [6]:
if mask_file == 'biomes_mask_05.nc':
    maskvar = 'sftlf'
    mask_values  = [0,1,2,3,4,5,6,7,8,9,10]         
    outdirname = 'biomes_regs'   
elif mask_file == 'ipcc_regions_05.nc':
    maskvar = 'regionID'
    mask_values  = [20,47,48,49,50,51,52,53,54,55,56,57,58]      
    outdirname = 'ipcc_regs'    
nregions = len(mask_values)

fmask= f'{path2mask}/{mask_file}'
maskara = read_mask(fmask, maskvar)
if fmask!='ipcc_regions_05.nc':
    region_ID = np.flipud(maskara)

Output folder: 

In [7]:
path2out  = f'{root}/data/data_txt/{resolution}/regional/{outdirname}/{varname}/{season}/'
if not os.path.exists(path2out):
    os.makedirs(path2out)

Read the list of the models to be processed:

In [ ]:
# Loop over seasons
for ssp in ssps:
    print("Working on the ssp: " + ssp)
    
    fname = [f'{path2data}/CMIP6_historical_{varname}_{season}.nc',
             f'{path2data}/CMIP6_{ssp}_{varname}_{season}.nc']

    var, lon, lat, time, dates = read_nc(fname, varname, True)

    # Define n dimensions
    nlats  = len(lat)
    nlons  = len(lon)
    ntimes = len(dates)

    # Read years
    years =np.empty(ntimes)
    for i in range(len(dates)): years[i]=dates[i].year 

    # Calculate weights
    xx, yy  = np.meshgrid(lon, lat)
    weights = np.cos(np.deg2rad(yy))

    # Allocate arrays
    run_ntimes = int(ntimes - avg_period - ref_ntimes + 1)
    delta_mean = np.empty([run_ntimes, nmembers, nlats, nlons])
    mask_4D    = np.empty(delta_mean.shape, dtype=bool)

    # Calculate mean for the reference period
    ref_data = var[0:ref_ntimes,:,:,:]
    ref_mean = np.nanmean(ref_data,axis=(0))
    mask_ref = np.empty(ref_data.shape, dtype=bool)

    # Calculate mean anomaly with respect to the reference period
    for t in np.arange(0, run_ntimes):
        it = t + ref_ntimes
        delta_mean[t,:,:,:] = np.nanmean(var[it:(it + avg_period-1), :, :, :], axis=(0)) - ref_mean

    # Masking regions
    for k in range(nregions):
        print("        Working on the region: " + str(mask_values[k])) 
        mask_value  = mask_values[k]
        mask_2D     = region_ID[:].data != mask_value

        # Masking weights
        masked_weights = ma.masked_array(weights, mask_2D) 

        # Creating 4D mask to mask 4D var
        for i in range(ref_ntimes):
            for j in range(nmembers):
                mask_ref[i,j,:,:] = mask_2D

        for i in range(run_ntimes):  
            for j in range(nmembers):
                mask_4D[i,j,:,:]  = mask_2D

        # Masking mean anomaly
        masked_ref_data = ma.masked_array(ref_data, mask_ref)
        masked_data     = ma.masked_array(delta_mean, mask_4D) 

        # Weighted mean anomaly
        delta_wght  = masked_data*masked_weights/np.nanmean(masked_weights)
        ref_wght    = masked_ref_data*masked_weights/np.nanmean(masked_weights)

        # Calculating spatial mean anomaly
        mean_delta_2D  = np.round(np.nanmean(delta_wght, axis=(2,3)),3)
        mean_reference = np.round(np.nanmean(ref_wght, axis=(2,3)),3)

        # Check if the signal is robust
        ifrobust = np.empty([run_ntimes])
        robustness = np.empty([run_ntimes, nmembers])

        for i in range(run_ntimes):
            plus  = 100.*float(np.count_nonzero(mean_delta_2D[i,:] >= 0))/float(nmembers)
            minus = 100.*float(np.count_nonzero(mean_delta_2D[i,:] < 0))/float(nmembers)              

            for j in range(nmembers):
                robustness[i,j]=robust(mean_reference[:,j], mean_delta_2D[i,j], avg_period)

            if (100.*float(np.count_nonzero(robustness[i,:] == 1.))/float(nmembers)) >= 66. and max(plus,minus) >= 80.:
                ifrobust[i]=1.
            elif (100.*float(np.count_nonzero(robustness[i,:] == 1.))/float(nmembers)) >= 66. and max(plus,minus) < 80.:
                ifrobust[i]=2.
            else:
                ifrobust[i]=0.                        


        # Save the regional mean difference in a txt file            
        output_name = f'{varname}_{ssp}_reg{mask_value}_{season}'
        print("        Saving output to: " + output_name)
        np.savetxt(path2out + output_name + '.csv', mean_delta_2D, fmt='%10.3f')            
        np.savetxt(path2out + 'ifrobust_' + output_name + '.csv', ifrobust, fmt='%10.3f')